In [7]:
import json
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from PIL import Image
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, load_model

In [8]:
model = load_model('my_model.keras')

In [9]:
train_image_dir = '재활용품_분류_및_선별_데이터/01-1.정식개방데이터/Validation/01.원천데이터/VS_1.영상추출_01.금속캔_002.알루미늄캔'
train_label_dir = '재활용품_분류_및_선별_데이터/01-1.정식개방데이터/Validation/02.라벨링데이터/VL_1.영상추출_01.금속캔_002.알루미늄캔'
img_height, img_width = 150, 150  # 원하는 이미지 크기

In [10]:
images = []
image_labels = []

In [11]:
# 디렉토리 내의 JSON 파일 탐색
for json_file in os.listdir(train_label_dir):
    if json_file.endswith('.json'):
        json_path = os.path.join(train_label_dir, json_file)
        
        # JSON 파일에서 라벨 데이터 읽기
        with open(json_path, encoding='utf-8') as f:
            data = json.load(f)
        
        img_file = data['IMAGE_INFO']['FILE_NAME']
        img_path = os.path.join(train_image_dir, img_file)
        
        # 이미지 파일이 실제로 존재하는지 확인
        if os.path.exists(img_path) and img_file.endswith('.jpg'):
            try:
                img = Image.open(img_path)  # 이미지를 실제로 불러와 확인
                annotations = data.get('ANNOTATION_INFO', [])
                for annotation in annotations:
                    points = annotation.get('POINTS', [None])[0]  # POINTS 리스트의 첫 번째 항목을 사용
                    if points and all(isinstance(x, (int, float)) for x in points):
                        images.append(img_path)  # 중복 허용: 이미지 경로를 여러 번 추가
                        image_labels.append((img_path, points, annotation))   # 이미지 경로와 포인트를 함께 저장
                # 라벨 수 출력
                print(f"File: {json_file}, Labels: {len(annotations)}")
            except (IOError, SyntaxError) as e:
                print(f"Error loading image {img_path}: {e}")

File: 1002786@0_01002_220907_P1_T3__3063.json, Labels: 1
File: 1004113@0_01002_220907_P1_T3__0974.json, Labels: 1
File: 1004318@0_01002_220907_P1_T3__1413.json, Labels: 1
File: 1005048@0_01002_220907_P1_T3__1665.json, Labels: 1
File: 1008334@0_01002_220907_P1_T3__2946.json, Labels: 3
File: 1010524@0_01002_220906_P1_T3__0128.json, Labels: 5
File: 1010532@0_01002_220906_P1_T3__0286.json, Labels: 6
File: 1010540@0_01002_220906_P1_T3__0329.json, Labels: 6
File: 1010545@0_01002_220906_P1_T3__0332.json, Labels: 5
File: 1010549@0_01002_220906_P1_T3__0190.json, Labels: 4
File: 1010552@0_01002_220906_P1_T3__0211.json, Labels: 4
File: 1010567@0_01002_220906_P1_T3__0335.json, Labels: 4
File: 1010576@0_01002_220906_P1_T3__0213.json, Labels: 6
File: 1010611@0_01002_220906_P1_T3__0222.json, Labels: 5
File: 1010625@0_01002_220906_P1_T3__0319.json, Labels: 6
File: 1010627@0_01002_220906_P1_T3__0101.json, Labels: 8
File: 1010639@0_01002_220906_P1_T3__0308.json, Labels: 6
File: 1011055@0_01002_220906_P1

In [12]:
print(f"Number of images: {len(images)}")
print(f"Number of labels: {len(image_labels)}")

Number of images: 5492
Number of labels: 5492


In [13]:
original_size = (1920, 1080)
target_size = (img_height, img_width)

In [14]:
def scale_points(points, original_size, target_size):
    # 원본 크기와 목표 크기 비율 계산
    scale_x = target_size[0] / original_size[0]
    scale_y = target_size[1] / original_size[1]

    # 포인트 스케일링
    scaled_points = [points[0] * scale_x, points[1] * scale_y, points[2] * scale_x, points[3] * scale_y]
    return scaled_points

In [15]:
def crop_image(img_path, points):
    img = Image.open(img_path)
    left, top, width, height = points
    right = left + width
    bottom = top + height

    # 포인트 유효성 검사
    img_width, img_height = img.size
    if left < 0 or top < 0 or right > img_width or bottom > img_height:
        raise ValueError(f"Invalid points for cropping: {points}")

    cropped_img = img.crop((left, top, right, bottom))
    return cropped_img

In [16]:
print(images[:1])

['재활용품_분류_및_선별_데이터/01-1.정식개방데이터/Validation/01.원천데이터/VS_1.영상추출_01.금속캔_002.알루미늄캔\\1002786@0_01002_220907_P1_T3__3063.jpg']


In [17]:
# image_labels 리스트의 구조 확인
print(image_labels[:1])  # 리스트의 처음 5개 항목 출력

[('재활용품_분류_및_선별_데이터/01-1.정식개방데이터/Validation/01.원천데이터/VS_1.영상추출_01.금속캔_002.알루미늄캔\\1002786@0_01002_220907_P1_T3__3063.jpg', [681.7559621451105, 855.7939760252366, 246.45368328075688, 222.7190309148266], {'ID': '1', 'CLASS': '금속캔', 'DETAILS': '알루미늄캔', 'DAMAGE': '찌그러짐', 'DIRTINESS': '오염없음', 'COVER': '없음', 'TRANSPARENCY': '불투명', 'SHAPE': '뚱뚱한 원통형', 'SHAPE_TYPE': 'BOX', 'POINTS': [[681.7559621451105, 855.7939760252366, 246.45368328075688, 222.7190309148266]]})]


In [18]:
train_images = []
train_labels = []
failed_images = []

In [19]:
# 이미지 크롭 및 배열로 변환
for img_path, points, annotation in image_labels:
    try:
        # 포인트 데이터의 길이 확인
        if len(points) != 4:
            raise ValueError(f"Points data does not contain 4 elements: {points}")

        scaled_points = scale_points(points, original_size, target_size)
        
        # 포인트 유효성 검사
        if any(p < 0 for p in scaled_points):
            raise ValueError(f"Scaled points have negative values: {scaled_points}")
        if scaled_points[0] + scaled_points[2] > original_size[0] or scaled_points[1] + scaled_points[3] > original_size[1]:
            raise ValueError(f"Scaled points exceed image boundaries: {scaled_points}")

        cropped_img = crop_image(img_path, scaled_points)
        cropped_img = cropped_img.resize(target_size)
        img_array = img_to_array(cropped_img)
        img_array = img_array / 255.0  # 정규화
        train_images.append(img_array)
        train_labels.append(annotation['CLASS'])  # 라벨 추가
    except Exception as e:
        print(f"Failed to process image {img_path} with error: {e}")
        # 실패한 이미지도 원본 상태로 추가
        try:
            img = Image.open(img_path)
            img = img.resize(target_size)
            img_array = img_to_array(img)
            img_array = img_array / 255.0  # 정규화
            train_images.append(img_array)
            train_labels.append(annotation['CLASS'])
        except Exception as inner_e:
            print(f"Failed to load original image {img_path} with error: {inner_e}")
            failed_images.append((img_path, points, str(inner_e)))


Failed to process image 재활용품_분류_및_선별_데이터/01-1.정식개방데이터/Validation/01.원천데이터/VS_1.영상추출_01.금속캔_002.알루미늄캔\1011262@0_01002_220906_P1_T3__2336.jpg with error: Points data does not contain 4 elements: [943.8139914893617, 128.79022978723404]
Failed to process image 재활용품_분류_및_선별_데이터/01-1.정식개방데이터/Validation/01.원천데이터/VS_1.영상추출_01.금속캔_002.알루미늄캔\1011267@0_01002_220906_P1_T3__1962.jpg with error: Points data does not contain 4 elements: [902.0410400680851, 143.88260629787237]
Failed to process image 재활용품_분류_및_선별_데이터/01-1.정식개방데이터/Validation/01.원천데이터/VS_1.영상추출_01.금속캔_002.알루미늄캔\1011278@0_01002_220906_P1_T3__2252.jpg with error: Points data does not contain 4 elements: [632.5803574468085, 710.9865191489362]
Failed to process image 재활용품_분류_및_선별_데이터/01-1.정식개방데이터/Validation/01.원천데이터/VS_1.영상추출_01.금속캔_002.알루미늄캔\1011285@0_01002_220906_P1_T3__1889.jpg with error: Points data does not contain 4 elements: [1050.9488149787235, 832.4587914893616]
Failed to process image 재활용품_분류_및_선별_데이터/01-1.정식개방데이터/Validation/01.원

In [20]:
print(len(train_images))
print(len(failed_images))
print(len(train_labels))

5492
0
5492


In [21]:
print(len(image_labels), image_labels[0])

5492 ('재활용품_분류_및_선별_데이터/01-1.정식개방데이터/Validation/01.원천데이터/VS_1.영상추출_01.금속캔_002.알루미늄캔\\1002786@0_01002_220907_P1_T3__3063.jpg', [681.7559621451105, 855.7939760252366, 246.45368328075688, 222.7190309148266], {'ID': '1', 'CLASS': '금속캔', 'DETAILS': '알루미늄캔', 'DAMAGE': '찌그러짐', 'DIRTINESS': '오염없음', 'COVER': '없음', 'TRANSPARENCY': '불투명', 'SHAPE': '뚱뚱한 원통형', 'SHAPE_TYPE': 'BOX', 'POINTS': [[681.7559621451105, 855.7939760252366, 246.45368328075688, 222.7190309148266]]})


In [22]:
# 각 라벨을 원-핫 인코딩하고 하나의 벡터로 결합하는 함수
def encode_and_combine_labels(label_lists):
    encoded_labels = []
    for labels in label_lists:
        unique_labels = set(labels)
        label_to_int = {label: i for i, label in enumerate(unique_labels)}
        integer_labels = [label_to_int[label] for label in labels]
        encoded_label = to_categorical(integer_labels, num_classes=len(unique_labels))
        encoded_labels.append(encoded_label)
    return np.hstack(encoded_labels)
    
# 모든 라벨 데이터를 리스트로 준비
label_lists = [
    [label_info[2]['CLASS'] for label_info in image_labels],  # label_info[2]는 라벨 딕셔너리
    [label_info[2]['DETAILS'] for label_info in image_labels],
    [label_info[2]['DAMAGE'] for label_info in image_labels],
    [label_info[2]['DIRTINESS'] for label_info in image_labels],
    [label_info[2]['COVER'] for label_info in image_labels],
    [label_info[2]['TRANSPARENCY'] for label_info in image_labels],
    [label_info[2]['SHAPE'] for label_info in image_labels],
    [label_info[2]['SHAPE_TYPE'] for label_info in image_labels]
]

# 라벨 인코딩 및 결합
all_labels = encode_and_combine_labels(label_lists)

In [23]:
print(len(train_images))
print(len(all_labels))

5492
5492


In [24]:
# 데이터셋 분할
train_images, val_images, train_labels, val_labels = train_test_split(train_images, all_labels, test_size=0.2, shuffle=True)

train_labels = np.argmax(train_labels, axis=1)
val_labels = np.argmax(val_labels, axis=1)

In [25]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense


# 모델 정의 및 컴파일
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')  # 클래스 수에 따라 변경
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',  # 다중 클래스 분류 손실 함수
              metrics=['accuracy'])

D:\EZEN\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [26]:
import numpy as np

# 리스트 형태의 데이터를 numpy 배열로 변환
train_images = np.array(train_images)
train_labels = np.array(train_labels)
val_images = np.array(val_images)
val_labels = np.array(val_labels)

# 데이터 크기 확인
print(train_images.shape, train_labels.shape)
print(val_images.shape, val_labels.shape)


(4393, 150, 150, 3) (4393,)
(1099, 150, 150, 3) (1099,)


In [27]:
history = model.fit(train_images, train_labels, epochs=10, validation_data=(val_images, val_labels))

Epoch 1/10
138/138 ━━━━━━━━━━━━━━━━━━━━ 35s 241ms/step - accuracy: 0.9628 - loss: 0.0924 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/10
138/138 ━━━━━━━━━━━━━━━━━━━━ 34s 245ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/10
138/138 ━━━━━━━━━━━━━━━━━━━━ 30s 219ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/10
138/138 ━━━━━━━━━━━━━━━━━━━━ 32s 232ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/10
138/138 ━━━━━━━━━━━━━━━━━━━━ 37s 270ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 6/10
138/138 ━━━━━━━━━━━━━━━━━━━━ 32s 230ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 7/10
138/138 ━━━━━━━━━━━━━━━━━━━━ 31s 226ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 8/10
138/138 ━━━━━━━━━━━━

In [28]:
# 모델 저장
model.save('my_model_updated.keras')